In [1]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchjpeg import dct
from PIL import Image
import seaborn as sns
import numpy as np
import torch
import math
import cv2
import os

/home/jianming/conda/envs/pica37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
block_size = 4
total_frequency_component = block_size * block_size

overall_img_path_list = []
path_prefix = "/home/jianming/work/multiface/dataset/m--20180227--0000--6795937--GHS/unwrapped_uv_1024/"
all_dir = os.listdir(path_prefix)
for sgl_dir in all_dir:
    path_average = os.path.join(path_prefix + sgl_dir, "average")
    overall_img_path_list.append(os.path.join(path_average, os.listdir(path_average)[0]))

overall_img_path_list2 = []
path_prefix2 = "/scratch1/jianming/multiface/dataset/m--20180226--0000--6674443--GHS/unwrapped_uv_1024/"
all_dir = os.listdir(path_prefix2)
for sgl_dir in all_dir:
    path_average2 = os.path.join(path_prefix2 + sgl_dir, "average")
    overall_img_path_list2.append(os.path.join(path_average2, os.listdir(path_average2)[0]))

transform = transforms.Compose([
        transforms.ToTensor()
        ])

downsample_components_list = []
for img_path in overall_img_path_list:
    image = Image.open(img_path).convert('RGB')
    x = transform(image).unsqueeze(0)
    x = (x + 1) / 2 * 255
    assert(x.shape[1] == 3, "Wrong input, Channel should equals to 3")
    original_in_img = dct.to_ycbcr(x)  # comvert RGB to YCBCR
    original_in_img -= 128
    downsample_components_list.append(original_in_img)

downsample_components_list2 = []
for img_path in overall_img_path_list2:
    image2 = Image.open(img_path).convert('RGB')
    x = transform(image2).unsqueeze(0)
    x = (x + 1) / 2 * 255
    assert(x.shape[1] == 3, "Wrong input, Channel should equals to 3")
    original_in_img2 = dct.to_ycbcr(x)  # comvert RGB to YCBCR
    original_in_img2 -= 128
    downsample_components_list2.append(original_in_img2)

downsample_components_overall = downsample_components_list + downsample_components_list2
# L2 norm among highest frequency components after BDCT decomposition
num_images = len(downsample_components_overall)

l2_norm_expression_list_overall = np.zeros((len(downsample_components_overall), len(downsample_components_overall)))
for i in range(num_images):
    for j in range(num_images):
        l2_norm_expression_list_overall[i][j] = np.linalg.norm(downsample_components_overall[i] - downsample_components_overall[j])
np.save(f"l2_norm_in_img_component.npy", l2_norm_expression_list_overall)

<>:27: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:37: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:27: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:37: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:27: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:37: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/tmp/ipykernel_671597/536562697.py:27: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(x.shape[1] == 3, "Wrong input, Channel should equals to 3")
/tmp/ipykernel_671597/536562697.py:37: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(x.shape[1] == 3, "Wrong input, Channel should equals to 3")


In [6]:
print(l2_norm_expression_list_overall.shape)
print(np.max(l2_norm_expression_list_overall))
max_l2_norm = np.max(l2_norm_expression_list_overall)

(131, 131)
13191.1337890625


In [9]:
# divide overall l2 norm into 10 equally splitted pieces and then fix the PSR at 0.25 -> obtain the final reconstructured L2 norm.
def calculate_prior_sr(target_l2norm, l2_norm_expression_list_overall):
    overall_category = l2_norm_expression_list_overall.shape[0]

    per_list_statistics = []
    for i in range(l2_norm_expression_list_overall.shape[0]):
        per_list_statistics.append(np.where(l2_norm_expression_list_overall[i,:] < target_l2norm))

    selective_list= []
    for per_row in per_list_statistics:
        selective_list.append(len(per_row[0]))
    
    pri_sr = np.max(selective_list)/overall_category

    return pri_sr

pri_sr_list = []
for i in range(100):
    target_l2norm = max_l2_norm * i / 100
    
    # calculate prior successful rate 
    pri_sr = calculate_prior_sr(target_l2norm, l2_norm_expression_list_overall)
    pri_sr_list.append(pri_sr)

print(pri_sr_list)

[0.0, 0.007633587786259542, 0.007633587786259542, 0.007633587786259542, 0.007633587786259542, 0.007633587786259542, 0.007633587786259542, 0.015267175572519083, 0.015267175572519083, 0.015267175572519083, 0.022900763358778626, 0.030534351145038167, 0.04580152671755725, 0.08396946564885496, 0.13740458015267176, 0.19083969465648856, 0.24427480916030533, 0.2748091603053435, 0.32061068702290074, 0.33587786259541985, 0.35877862595419846, 0.37404580152671757, 0.3893129770992366, 0.4122137404580153, 0.42748091603053434, 0.44274809160305345, 0.45038167938931295, 0.46564885496183206, 0.4732824427480916, 0.48091603053435117, 0.4961832061068702, 0.5038167938931297, 0.5038167938931297, 0.5038167938931297, 0.5038167938931297, 0.5038167938931297, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893, 0.5114503816793893

In [14]:
selected_pri_sr_list = np.array(pri_sr_list)[np.where(np.array(pri_sr_list) < 0.2)[0]]
selected_l2_norm_list = []
for i in range(100):
    target_l2norm = max_l2_norm * i / 100
    if len(selected_l2_norm_list) <= len(selected_pri_sr_list):
        selected_l2_norm_list.append(target_l2norm)
print(selected_l2_norm_list)

[0.0, 131.911337890625, 263.82267578125, 395.734013671875, 527.6453515625, 659.556689453125, 791.46802734375, 923.379365234375, 1055.290703125, 1187.202041015625, 1319.11337890625, 1451.024716796875, 1582.9360546875, 1714.847392578125, 1846.75873046875, 1978.670068359375, 2110.58140625]


In [53]:
import numpy as np
from scipy.special import rel_entr

def kl_divergence(p, q):
    """
    Calculate the Kullback-Leibler (KL) divergence between two probability distributions.
    
    Args:
    p (array-like): The true probability distribution (must be a valid probability distribution).
    q (array-like): The estimated probability distribution (must be a valid probability distribution).
    
    Returns:
    float: The KL divergence between distributions p and q.
    """
    p = np.asarray(p, dtype=np.float)
    q = np.asarray(q, dtype=np.float)
    
    # Ensure that neither p nor q contains zero values
    p = np.clip(p, 1e-10, 1)
    q = np.clip(q, 1e-10, 1)
    
    # Calculate KL divergence
    return np.sum(rel_entr(p, q))

# Example usage
p = [0.2]
q = [0.1]

kl_div = kl_divergence(p, q)
print(f"KL Divergence: {kl_div}")

def calculate_edge_kl_divergence(prior, mi):
    posterior = prior
    posterior_pre = prior
    for test_val in range(100):
        posterior = test_val*(1-prior)/100 + prior
        if kl_divergence(posterior_pre, prior) < mi and kl_divergence(posterior, prior) >= mi:
            return posterior
    return 1

def calculate_mi_bound(prior, posterior_target):
    posterior = posterior_target
    for test_val in range(10000):
        mi = test_val / 1000
        mi_next = (test_val + 1) / 1000

        if kl_divergence(posterior, prior) < mi_next and kl_divergence(posterior, prior) >= mi:
            return mi
    return -1

def obtain_mi_list_under_various_prior_rate(selected_pri_sr_list, posterior_target):
    selected_mi_list = []
    for pri_sr in selected_pri_sr_list:
        selected_mi_list.append(calculate_mi_bound(pri_sr, posterior_target=posterior_target))
    return selected_mi_list

KL Divergence: 0.13862943611198905


/home/jianming/conda/envs/pica37/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ipykernel import kernelapp as app
/home/jianming/conda/envs/pica37/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [52]:
# obtain the mutual information list of mutual information for getting a specific PSR
selected_mi_list = obtain_mi_list_under_various_prior_rate(selected_pri_sr_list, 0.25)

/home/jianming/conda/envs/pica37/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ipykernel import kernelapp as app
/home/jianming/conda/envs/pica37/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:
def calculate_covariance_matrix(number_files, batch_size, captured_data_list):
    captured_texture_avg_data = np.zeros(((number_files-1)*batch_size, 256))

    for i in range(number_files-1):
        texture_avg_file_list = f"{captured_data_list}/z_{i+1}.pth"
        captured_texture_avg = torch.load(texture_avg_file_list).to("cpu")
        captured_texture_avg_data[i*batch_size:(i+1)*batch_size] = captured_texture_avg.detach().numpy()
    covariance_matrix_texture_avg = np.cov(captured_texture_avg_data, rowvar=False)
    return covariance_matrix_texture_avg, captured_texture_avg_data

def calculate_covariance_matrix_outsource_latent_code(number_files, batch_size, captured_data_list):
    captured_texture_avg_data = np.zeros(((number_files-1)*batch_size, 256))

    for i in range(number_files-1):
        texture_avg_file_list = f"{captured_data_list}/texture_avg_{i+1}.pth"
        captured_texture_avg = torch.load(texture_avg_file_list).to("cpu")
        captured_texture_avg_data[i*batch_size:(i+1)*batch_size] = captured_texture_avg.detach().numpy()
    covariance_matrix_texture_avg = np.cov(captured_texture_avg_data, rowvar=False)
    return covariance_matrix_texture_avg

def plot_eigenvalue_covariance_matrix(covariance_matrix_texture_avg, img_save_path):
    U, s, V = np.linalg.svd(covariance_matrix_texture_avg)

    X = [i for i in range(covariance_matrix_texture_avg.shape[0])]

    # Data for plotting
    SMALL_SIZE = 20
    MEDIUM_SIZE = 22
    BIGGER_SIZE = 24

    plt.figure(figsize=(9, 6))
    plt.plot(X, s)
    plt.title('SVD decomposition of Cov(Z): (Z vector)', fontsize=SMALL_SIZE)

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.savefig(f'{img_save_path}.pdf', bbox_inches="tight", transparent=True) 
    
# calculate noise of each dimensions
def generate_noise_covariance(mutual_info_bound, s_cov):
    overall_result_cov = 0
    for ele in s_cov:
        overall_result_cov = overall_result_cov + np.sqrt(ele)
    print()
    print(f"sum of sqrt={overall_result_cov}")
    print(f"mutual information = 1 -- noise std variance={overall_result_cov/np.sqrt(2)}")
    noise_variance = np.zeros(256)
    for i in range(256):
        print(f"s_cov[i]={s_cov[i]}, overall_result_cov={overall_result_cov}, div={(2 * mutual_info_bound)}")
        noise_variance[i] =  (np.sqrt(s_cov[i]) * overall_result_cov) / (2 * mutual_info_bound)
    return noise_variance

In [ ]:
# calculate the amount of injected noise under different number of outsourced components
## Step 1: Obtain the outsourced latent code and then calculate overall amount of noises in need of injection into the framework.
## Calculate the covariance of various different horizontal partitioning setup
captured_data_list = "asd"
number_files = 16
batch_size = 1
calculate_covariance_matrix()

## Step 2: calculate the amount of noises that should be injected into the outsourced data
for mi in selected_mi_list:
    print()